In [249]:
import pandas as pd

In [260]:
facilities = pd.read_csv("./large/facilities.csv")
covenants = pd.read_csv("./large/covenants.csv")
loans = pd.read_csv("./large/loans.csv")
#loans.head()

In [261]:
# Grouping the state column to a list
covenantState = covenants.groupby('facility_id').apply(lambda x: list(x.banned_state))
covenantState = pd.DataFrame(covenantState, columns = ["Banned"])
covenantState['facility_id'] = covenantState.index
# Grouping the max_default_likelihood
covenantLikelihood = covenants.groupby('facility_id').max()
covenantLikelihood = pd.DataFrame(covenantLikelihood, columns = ["max_default_likelihood"])
covenantLikelihood['facility_id'] = covenantLikelihood.index

# Merging covenantState and covenantLikelihood
facilityCovenant = pd.merge(facilities,covenantState[['Banned']],left_on='id',right_on='facility_id',how='left')
facilityCovenant = pd.merge(facilityCovenant,covenantLikelihood[['max_default_likelihood']],left_on='id',right_on='facility_id',how='left')
facilityCovenant = facilityCovenant.sort_values('interest_rate')
facilityCovenant = facilityCovenant.reset_index(drop=True)
#facilityCovenant

In [262]:
assignDictionary = {}
for loanIndex,loanRow in loans.iterrows():
    maxExpectedYield = 0
    for fcIndex,fcRow in facilityCovenant.iterrows():
        expectedYield = 0
        if(loanRow['amount'] <= fcRow['amount']):
            if(loanRow['default_likelihood']<=fcRow['max_default_likelihood']):
                if(loanRow['state'] not in fcRow['Banned']):
                    expectedYield = (1 - loanRow['default_likelihood']) * loanRow['interest_rate'] * loanRow['amount'] - loanRow['default_likelihood'] * loanRow['amount'] - fcRow['interest_rate'] * loanRow['amount']
                    if(expectedYield > maxExpectedYield):
                        maxExpectedYield = expectedYield
                        assignDictionary[loanRow['id']] = [fcRow['id'],expectedYield,fcRow['amount']-loanRow['amount']]
    #print(assignDictionary)
    facilityCovenant['amount'][(facilityCovenant['id'] == assignDictionary[loanRow['id']][0])] = assignDictionary[loanRow['id']][2]
#print(assignDictionary)

D:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [263]:
yieldDf = pd.DataFrame(assignDictionary)
yieldDf = yieldDf.transpose()
#print(yieldDf)
yieldDf = round(yieldDf.groupby(0).sum())
yieldDf = yieldDf.reset_index()
yieldDf.columns = ['facility_id','yield','amount']
yieldDf = yieldDf[['facility_id','yield']]
#yieldDf

In [264]:
assignDf = pd.DataFrame(assignDictionary)
assignDf = assignDf.transpose()
assignDf = assignDf.reset_index()
assignDf.columns = ['loan_id','facility_id','yeild','amount']
assignDf = assignDf[['loan_id','facility_id']]
#assignDf

In [255]:
assignDf.to_csv('./large/assignment.csv', sep=',',index=False)
yieldDf.to_csv('./large/yield.csv', sep=',',index=False)